<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/LSTM_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bidirectional LSTM Hate Speech Classifier

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import Bidirectional # new! 
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt
from keras.preprocessing.text import one_hot
from tensorboard.plugins.hparams import api as hp

In [5]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# directory name 
input_dir = '/content/drive/My Drive/HLT/clean_dataset_training/' 

# Spec
pd.set_option("display.max_colwidth", None)

In [7]:
tsv_file = open(input_dir+"training_dataset.csv")

dataset = pd.read_csv(tsv_file,sep=',')

### Vector-space embedding: 

In [8]:
p_train=0.85 # percentage of training set 

n_dim = 64 
n_unique_words = 20000 
max_length = 100 # doubled!
pad_type = trunc_type = 'pre'

# training 
batch_size = 64

#### Preprocess data 

In [9]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

all_words = []
for index, row in dataset.iterrows():
  tokenize_word = word_tokenize(row["text"])
  for word in tokenize_word:
      all_words.append(word)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
unique_words = set(all_words)
print(len(unique_words))

24773


In [11]:
embedded_sentences = [one_hot(row["text"], n_unique_words) for i,row in dataset.iterrows()]
print(len(embedded_sentences))

6837


In [12]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(dataset.get("text"), key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [125]:
dataset1 = dataset
dataset1 = dataset1.drop(['text', 'id', 'hs', 'stereotype','tokens', 'hashtags'], axis=1)
dataset1

,text_length,#C-L words,#?!,"#.,",#bad_words,%bad_words
0,120,10,0,5,0,0
1,101,0,0,0,1,6
2,86,8,0,1,3,25
3,118,0,0,2,0,0
4,138,0,1,1,1,4
...,...,...,...,...,...,...
6832,285,2,0,4,0,0
6833,277,0,2,3,0,0
6834,233,0,0,4,0,0
6835,206,2,0,2,0,0


In [132]:
dataset1.values[0:int(dataset.shape[0]*p_train)]

array([[120,  10,   0,   5,   0,   0],
       [101,   0,   0,   0,   1,   6],
       [ 86,   8,   0,   1,   3,  25],
       ...,
       [276,   2,   0,   8,   0,   0],
       [190,   0,   0,   5,   0,   0],
       [292,   4,   0,   6,   0,   0]])

In [130]:
x_train=embedded_sentences[0:int(dataset.shape[0]*p_train)]
x_train_extra = dataset1.values[0:int(dataset.shape[0]*p_train)]
y_train=dataset[['hs']][0:int(dataset.shape[0]*p_train)]
x_valid=embedded_sentences[int(dataset.shape[0]*p_train):]
x_valid_extra = dataset1.values[int(dataset.shape[0]*p_train):]
y_valid=dataset[['hs']][int(dataset.shape[0]*p_train):]

In [133]:
x_train = pad_sequences(x_train, maxlen=max_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_length, padding=pad_type, truncating=trunc_type, value=0)

In [134]:
input_train = {"text": x_train, "other": x_train_extra}
input_val   = {"text": x_valid, "other": x_valid_extra}

In [48]:
max_sent = 0 

In [49]:
import ast 

def comment_length(text):
    global max_sent 
    text = ast.literal_eval(text)
    if len(text)>max_sent: 
      max_sent = len(text)

In [17]:
dataset['tokens'].apply(comment_length)
print(max_sent)

119


In [18]:
x_train.shape

(5811, 100)

### Design grid search parameters

In [19]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [20]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128, 256, 512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.7))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['nadam']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [138]:
def train_test_model(hparams):
  lstml1_in = tf.keras.layers.Input(name="text", shape =(max_length,))
  lstml1_embedded = tf.keras.layers.Embedding(n_unique_words, n_dim, input_length=max_length)(lstml1_in)
  lstml1_sd = tf.keras.layers.SpatialDropout1D(hparams[HP_DROPOUT])(lstml1_embedded)
  lstml1_bd1 = tf.keras.layers.Bidirectional(LSTM(hparams[HP_NUM_UNITS], dropout = hparams[HP_DROPOUT], return_sequences=True))(lstml1_sd)
  lstml1_bd2 = tf.keras.layers.Bidirectional(LSTM(hparams[HP_NUM_UNITS], dropout = hparams[HP_DROPOUT]))(lstml1_bd1)
  
  other_in = tf.keras.layers.Input(name="other", shape =(6,))

  lconcat = tf.keras.layers.Concatenate(axis=1)([lstml1_bd2, other_in])
  dense1_layer = Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.007))(lconcat)
  lstml1_out = tf.keras.layers.Dense(1, activation='sigmoid')(dense1_layer)


  model = tf.keras.Model(inputs = [lstml1_in, other_in], outputs = lstml1_out)
  
  model.summary()
  
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='binary_crossentropy',
      metrics=['accuracy'],
  )
  
  model.fit(input_train, y_train, batch_size=batch_size, validation_data=(input_val, y_valid), epochs=10) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(input_val, y_valid)
  return accuracy

In [136]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [139]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 64, 'dropout': 0.2, 'optimizer': 'nadam'}
Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_34 (Embedding)        (None, 100, 64)      1280000     text[0][0]                       
__________________________________________________________________________________________________
spatial_dropout1d_34 (SpatialDr (None, 100, 64)      0           embedding_34[0][0]               
__________________________________________________________________________________________________
bidirectional_68 (Bidirectional (None, 100, 128)     66048       spatial_dropout1d_34[0][0]       
_________

KeyboardInterrupt: ignored